In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pyodbc
from urllib.parse import urljoin
from datetime import datetime, timedelta
import re
import os


In [3]:
url = "https://www.jkb.com/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
response = requests.get(url, headers=headers)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")


In [4]:
folder_name = "Jordan Kuwait Bank Images"
os.makedirs(folder_name, exist_ok=True)

banner_slides = soup.select(".banner_sec_slide img")
invest_banners = soup.select(".invest_banner img")
all_images = banner_slides + invest_banners

all_images = banner_slides + invest_banners


In [6]:
images = []


In [7]:
for img_tag in all_images:
    img_src = img_tag.get("src", "").strip()
    
    if img_src and img_src != "/":
        # معالجة الرابط: لو ما فيه https نكمل عليه رابط الموقع
        if img_src.startswith("/"):
            full_img_url = "https://www.jkb.com" + img_src
        else:
            full_img_url = img_src
        
        # استخراج اسم الصورة من الرابط
        img_name = img_src.split("/")[-1].split("?")[0]

        # استخراج رابط المقال (Article Link)
        article_link = ""
        for parent in img_tag.find_parents():
            # الحالة 1: لو فيه data-url
            if parent.has_attr("data-url"):
                article_link = urljoin(url, parent["data-url"])
                break
            # الحالة 2: لو فيه a.stretched-link
            stretched_a = parent.find("a", class_="stretched-link")
            if stretched_a and stretched_a.has_attr("href"):
                article_link = urljoin(url, stretched_a["href"])
                break

        # مسار حفظ الصورة
        img_path = os.path.join(folder_name, img_name)
        
        # تحميل الصورة
        try:
            img_response = requests.get(full_img_url)
            img_response.raise_for_status()
            with open(img_path, "wb") as f:
                f.write(img_response.content)
            print(f"✔️ تم تحميل الصورة: {img_name}")
            
            # إضافة بيانات للصورة
            images.append({
                "Bank_Name": "Jordan Kuwait Bank",
                "Image_Name": img_name,
                "Image_Link": full_img_url,
                "Article_Link": article_link
            })
            
        except Exception as e:
            print(f"❌ فشل تحميل الصورة: {full_img_url}, السبب: {e}")


✔️ تم تحميل الصورة: website2000-550-en-2000x550.png
✔️ تم تحميل الصورة: website768-570-en-2-768x570.png
✔️ تم تحميل الصورة: website2000-550-en-2000x550.png
✔️ تم تحميل الصورة: website768-570-en-768x570.png
✔️ تم تحميل الصورة: websitehome-page-2000-550-en-2000x550.png
✔️ تم تحميل الصورة: website768-570-en-768x570.png
✔️ تم تحميل الصورة: websitehome-page-2000-550-en-1-2000x550.png
✔️ تم تحميل الصورة: website768-570-en-1-768x570.png
✔️ تم تحميل الصورة: websitehome-page-2000-550-en-2-2000x550.png
✔️ تم تحميل الصورة: website768-570-en-2-768x570.png
✔️ تم تحميل الصورة: elieng-2862x1104.png
✔️ تم تحميل الصورة: website2862x1104-english1-2862x1104.png


In [11]:
df = pd.DataFrame(images)
df

,Bank_Name,Image_Name,Image_Link,Article_Link
0,Jordan Kuwait Bank,website2000-550-en-2000x550.png,https://www.jkb.com/uploads/2024/11/website200...,https://www.jkb.com/en/housing-loan-initiative...
1,Jordan Kuwait Bank,website768-570-en-2-768x570.png,https://www.jkb.com/uploads/2024/11/website768...,https://www.jkb.com/en/housing-loan-initiative...
2,Jordan Kuwait Bank,website2000-550-en-2000x550.png,https://www.jkb.com/uploads/2025/02/website200...,https://www.jkb.com/en/3-cash-back-campaign-wi...
3,Jordan Kuwait Bank,website768-570-en-768x570.png,https://www.jkb.com/uploads/2025/02/website768...,https://www.jkb.com/en/3-cash-back-campaign-wi...
4,Jordan Kuwait Bank,websitehome-page-2000-550-en-2000x550.png,https://www.jkb.com/uploads/2025/04/websitehom...,https://www.jkb.com/en/terms-and-conditions-fo...
5,Jordan Kuwait Bank,website768-570-en-768x570.png,https://www.jkb.com/uploads/2025/04/website768...,https://www.jkb.com/en/terms-and-conditions-fo...
6,Jordan Kuwait Bank,websitehome-page-2000-550-en-1-2000x550.png,https://www.jkb.com/uploads/2025/04/websitehom...,https://www.jkb.com/en/google-pay
7,Jordan Kuwait Bank,website768-570-en-1-768x570.png,https://www.jkb.com/uploads/2025/04/website768...,https://www.jkb.com/en/google-pay
8,Jordan Kuwait Bank,websitehome-page-2000-550-en-2-2000x550.png,https://www.jkb.com/uploads/2025/04/websitehom...,https://www.jkb.com/en/housing-loans-campaign-2
9,Jordan Kuwait Bank,website768-570-en-2-768x570.png,https://www.jkb.com/uploads/2025/04/website768...,https://www.jkb.com/en/housing-loans-campaign-2
